# $B_1$ and $B_2$ in model test and SI

In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score
import rolldecay.paper_writing as paper_writing



In [ ]:
df_ikeda = database.load(rolldecay_table_name='rolldecay_simplified_ikeda', limit_score=0.1, 
                             exclude_table_name='rolldecay_exclude')

df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.99, 
                             exclude_table_name='rolldecay_exclude')

In [ ]:
df_compare = pd.merge(left=df_rolldecay, right=df_ikeda, how='inner', left_index=True, right_index=True, suffixes=('','_SI'))

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_1',y='B_1_SI', ax=ax, style='.')

In [ ]:
fig,ax=plt.subplots()
df_compare.plot(x='B_2',y='B_2_SI', ax=ax, style='.')
ax.grid(True)

In [ ]:
run = df_rolldecay.sort_values(by='B_2').iloc[0]

In [ ]:
db = database.get_db()
db_run = db.session.query(Run).get(int(run.name))
df = database.load_run(db_run, save_as_example=False)

In [ ]:
df.plot(y='phi')

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
#scaler = ScaleFactorTransformer(scale_factor=db_run.model.scale_factor)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25))
offset_transformer = OffsetTransformer()

steps = [
    ('filter',lowpass_filter),
#    ('scaler',scaler),  # Is froude scaling a good idea??
    ('cutter', cutter), 
    #('offset_transformer',offset_transformer)
]
        
preprocessor = Pipeline(steps) # define the pipeline object.
preprocessor.fit(df)
X = preprocessor.transform(df)

In [ ]:
X.plot(y='phi')
X.plot(y='phi1d')

In [ ]:
estimator = EstimatorQuadraticB.load(**run, X=X)

In [ ]:
fig,ax=plt.subplots()
estimator.plot_fit(ax=ax)
estimator.plot_amplitude(ax=ax)

In [ ]:
fig,ax=plt.subplots()
estimator.plot_damping(ax=ax)

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
#scaler = ScaleFactorTransformer(scale_factor=db_run.model.scale_factor)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(6), phi_min=np.deg2rad(0.5))
offset_transformer = OffsetTransformer()

steps = [
    ('filter',lowpass_filter),
#    ('scaler',scaler),  # Is froude scaling a good idea??
    ('cutter', cutter), 
    #('offset_transformer',offset_transformer)
]
        
preprocessor = Pipeline(steps) # define the pipeline object.
preprocessor.fit(df)
X2 = preprocessor.transform(df)

In [ ]:
estimator2 = EstimatorQuadraticB(ftol=1e-09, maxfev=100000)
estimator2.fit(X2)

In [ ]:
fig,ax=plt.subplots()
estimator2.plot_fit(ax=ax)
estimator2.plot_amplitude(ax=ax)

In [ ]:
fig,ax=plt.subplots()
estimator2.plot_damping(ax=ax)

In [ ]:
run.project_path